In [33]:
# Cell 1 : Tensorboard management

%load_ext tensorboard
import datetime, os
%rm -rf logs/ # to clear previous logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


UsageError: Line magic function `%rm` not found.


In [34]:
# Cell 2 : Image data management

from keras.preprocessing.image import ImageDataGenerator

# Batch size
BS = 8

# Create image generators, with data augmentation
datagenerator = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.4,
                                   height_shift_range=0.4,
                                   brightness_range=[0.2,1],
                                   shear_range=0.3,
                                   zoom_range=[0.5,2],
                                   horizontal_flip=True,
                                   validation_split=0.2)

train_generator = datagenerator.flow_from_directory('dataset/train_images',
                                                  target_size=(200,200),
                                                  class_mode='categorical',
                                                  batch_size=BS,
                                                  shuffle=True,
                                                  color_mode='rgb',
                                                  subset='training')

validation_generator = datagenerator.flow_from_directory('dataset/train_images',
                                                        target_size=(200,200),
                                                        class_mode='categorical',
                                                        batch_size=BS,
                                                        shuffle=True,
                                                        color_mode='rgb',
                                                        subset='validation')


print("Data generators built successfully !")

Found 123 images belonging to 2 classes.
Found 30 images belonging to 2 classes.
Data generators built successfully !


In [35]:
# Cell 3 : Model creation and fitting

from keras.models import Sequential
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout,BatchNormalization,Activation
from keras.optimizers import SGD
import tensorflow as tf
import datetime

# Define training options
EPOCHS=300
lr = 0.01

# Define model
model = Sequential()
model.add(Conv2D(4,(3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(4,(3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(8,(3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(8,(3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(16,(3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(16,(3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(10,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2,activation='softmax'))

# Optimizer and model compilation
opt = SGD(learning_rate=lr)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

# Define the log directory for tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
H = model.fit(train_generator,validation_data=validation_generator,epochs=EPOCHS,callbacks=[tensorboard_callback])



Epoch 1/300
16/16 [==============================] - 5s 276ms/step - loss: 0.9143 - accuracy: 0.5772 - val_loss: 0.6772 - val_accuracy: 0.7333
Epoch 2/300
16/16 [==============================] - 4s 244ms/step - loss: 0.6740 - accuracy: 0.7154 - val_loss: 0.6647 - val_accuracy: 0.7333
Epoch 3/300
16/16 [==============================] - 4s 244ms/step - loss: 0.6510 - accuracy: 0.7154 - val_loss: 0.6543 - val_accuracy: 0.7333
Epoch 4/300
16/16 [==============================] - 4s 239ms/step - loss: 0.6408 - accuracy: 0.7154 - val_loss: 0.6457 - val_accuracy: 0.7333
Epoch 5/300
16/16 [==============================] - 4s 237ms/step - loss: 0.6183 - accuracy: 0.7154 - val_loss: 0.6219 - val_accuracy: 0.7333
Epoch 6/300
16/16 [==============================] - 4s 236ms/step - loss: 0.6102 - accuracy: 0.7154 - val_loss: 0.6049 - val_accuracy: 0.7333
Epoch 7/300
16/16 [==============================] - 4s 242ms/step - loss: 0.5668 - accuracy: 0.7154 - val_loss: 0.5847 - val_accuracy: 0.7333

In [36]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 896), started 2:24:31 ago. (Use '!kill 896' to kill it.)

In [3]:
# Cell 4 : Model saving, loading, and evaluation

import numpy as np
from keras.models import load_model
from sklearn.metrics import confusion_matrix

# Save trained model (uncomment to save newly trained model)

directory = "./models/"
filename = 'trained_model_fields_and_roads.h5'
#path = os.path.join(directory, filename)
#model.save(path)
#print('Trained model saved as %s' % path)

# Load trained model (uncomment if model is not freshly trained)
#model = load_model('./models/trained_model_fields_and_roads.h5')
#print("Trained model loaded successfully !")

# Evaluate the model
test_generator = datagenerator.flow_from_directory('dataset/test_images',
                                                 target_size=(200,200),
                                                 batch_size=10,
                                                 class_mode='categorical',
                                                 shuffle=False,
                                                 color_mode='rgb')

# Confusion matrix 

Y_pred = model.predict(test_generator, verbose=1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

# First line of confusion matrix : true "fields" labels
# Second line of confusion matrix : true "roads" labels
# First column of confusion matrix : predicted "fields" labels
# Second column of confusion matrix : predicted "roads" labels








ValueError: Unknown `data_format`: tf. Expected one of {'channels_first', 'channels_last'}